In [1]:
!pip install -q langchain_core langchain_ollama
!pip install -U "langchain[google-genai]"


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [39]:
import os
from access import Access

# os.environ['ANTHROPIC_API_KEY'] = Access.LANGCHAIN_API_KEY

In [ ]:
# print(os.getenv("ANTHROPIC_API_KEY"))

In [3]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 667b0c1932bc: 100% ▕██████████████████▏ 4.9 GB                         
pulling 948af2743fc7: 100% ▕██████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4: 100% ▕██████████████████▏  12 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 455f34728c9b: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [4]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(
    model="llama3.1",
    temperature=0.7,
    # max_tokens=1000
)

# from langchain_anthropic import ChatAnthropic

# chat_model = ChatAnthropic(
#   model="claude-3-sonnet-20240229",
#   temperature=0,
#   api_key="YOUR_ANTHROPIC_API_KEY"
# )

# from langchain.chat_models import init_chat_model

# model = init_chat_model(
#     "claude-sonnet-4-5-20250929",
#     # Kwargs passed to the model:
#     temperature=0.7,
#     timeout=30,
#     max_tokens=1000,
# )

In [5]:
chat_model.invoke("Tell me a joke about penguin!")

AIMessage(content='Why did the penguin take his credit card to the Antarctic?\n\nBecause he wanted to freeze his assets.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T13:28:06.375898Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13496573541, 'load_duration': 11580159916, 'prompt_eval_count': 18, 'prompt_eval_duration': 635004125, 'eval_count': 22, 'eval_duration': 1144876417, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--312e0c92-f841-4815-b1ae-e0e22ba6ba5f-0', usage_metadata={'input_tokens': 18, 'output_tokens': 22, 'total_tokens': 40})

The output shuold be in this format:
```
AIMessage(content="Here's a bear joke for you:\\n\\nWhy did the bear dissolve in water?\\nBecause it was a polar bear!")
```

In [6]:
# Another way of doing this
from langchain_core.messages import HumanMessage

chat_model.invoke([
    HumanMessage("Tell me a joke about bears!")
])

# Just the same but this one doesn't take simple string

AIMessage(content='Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T13:28:11.976963Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1700390292, 'load_duration': 102051375, 'prompt_eval_count': 17, 'prompt_eval_duration': 508642333, 'eval_count': 18, 'eval_duration': 973108417, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--a753a00e-8397-46d6-92de-84fc4f4d2852-0', usage_metadata={'input_tokens': 17, 'output_tokens': 18, 'total_tokens': 35})

Result:
```
AIMessage(content="Here's a bear joke for you:\\n\\nWhy did the bear bring a briefcase to work?\\nHe was a business bear!")
```

## Prompt Templates

We can also parameterize the input using prompt templates

In [7]:
from langchain_core.prompts import ChatPromptTemplate

joke_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class comedian."),
    ("human", "Tell me a joke about {topic}")
])

In [8]:
joke_prompt.invoke({"topic": "beets"})

ChatPromptValue(messages=[SystemMessage(content='You are a world class comedian.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about beets', additional_kwargs={}, response_metadata={})])

Result:
```
ChatPromptValue(messages=[
    SystemMessage(content='You are a world class comedian.'),
    HumanMessage(content='Tell me a joke about beets')
])
```

## Chaining

Both Prompt Template and Chat Model implement the `.invoke()` method because they are both instances of Runnables.

We can compose runnables into "chains" using the pipe(`|`) operator where we invoke the next step with the output of the previous one. Example:

In [9]:
chain = joke_prompt | chat_model

In [10]:
chain.invoke({"topic": "beets"})

AIMessage(content='Here\'s one:\n\n"Why did the beet go to therapy? Because it was feeling a little \'root\'-less! (ba-dum-tss)"', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T13:28:20.33072Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2381205125, 'load_duration': 99168208, 'prompt_eval_count': 29, 'prompt_eval_duration': 372763209, 'eval_count': 31, 'eval_duration': 1708107418, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--48d7d7c6-24af-491c-b99c-d99680c19e86-0', usage_metadata={'input_tokens': 29, 'output_tokens': 31, 'total_tokens': 60})

Result should be like:
```
AIMessage(content="Here's a beet joke for you:\\n\\nWhy did the beet blush? Because it saw the salad dressing!")
```

## Output Parser

Output Parser help us present the result as raw string

In [11]:
from langchain_core.output_parsers import StrOutputParser

str_chain = chain | StrOutputParser()

# equivalent to:
# str_chain = joke_prompt | chat_model | StrOutputParser()

In [12]:
str_chain.invoke({"topic": "kobe bryant"})

'Here\'s one:\n\n"You know, Kobe Bryant was known for being super competitive, even on the golf course. In fact, I heard he once challenged Tiger Woods to a game and said, \'I\'m gonna drain this putt, Tiger... and then I\'m gonna be the GOAT!\' But Tiger just smiled and said, \'Kobe, you can\'t be the Goat of the green – that\'s a sheep.\' (ba-dum-tss)"'

## Streaming

Advantage of composing chain with LCEL is the streaming experience.

In [13]:
for chunk in str_chain.stream({"topic": "beets"}):
    print(chunk, end="|")

Here|'s| one|:

|Why| did| the| beet| go| to| therapy|?

|Because| it| was| feeling| a| little| "|root|"-|less|!| (|get| it|?)|||

In [14]:
# chat_model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

chat_model = ChatOllama(
    model="llama3.1",
    temperature=0.7,
    # max_tokens=1000
)

chat_model.invoke("What is the current date?")

AIMessage(content='I\'m an AI, I don\'t have real-time access to the current date. However, I can suggest ways for you to find out the current date:\n\n1. Check your device\'s calendar or clock app.\n2. Search online for "current date" on a search engine like Google.\n3. Ask a voice assistant like Siri, Google Assistant, or Alexa.\n\nIf you need help with something specific related to dates, feel free to ask, and I\'ll do my best to assist you!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T14:43:01.233334Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11430341875, 'load_duration': 4915078500, 'prompt_eval_count': 16, 'prompt_eval_duration': 453954333, 'eval_count': 101, 'eval_duration': 5447284622, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--c1c6039f-66d1-4d04-b20b-5f1f70ebc71b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 101, 'total_tokens': 117})

In [30]:
from datetime import date

prompt = ChatPromptTemplate.from_messages([
    ("system", 'You know that the current date is "{current_date}".'),
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser()

chain.invoke({
    "question": "What is the current date?",
    "current_date": date.today()
})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: You know that the current date is \"2025-11-28\".\nHuman: What is the current date?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOllama] [1.16s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The current date is \"2025-11-28\".",
        "generation_info": {
          "model": "llama3.1",
          "created_at": "2025-11-28T02:43:28.599116Z",
          "done": true,
          "done_reason": "stop",
          "total_duration": 1156636334,
          "load_duration": 92243417,
          "prompt_eval_count": 36,
          "prompt_eval_duration": 21

'The current date is "2025-11-28".'

In [16]:
chat_model.invoke(
    "What was the Old Ship Saloon's total revenue in Q1 2023?"
 )

AIMessage(content="I don't have access to real-time data or specific financial information about the Old Ship Saloon. For the most accurate and up-to-date information, I recommend checking their official website, contacting them directly, or looking at publicly available financial reports if they are a publicly traded company.", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-27T14:58:18.020913Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4629176917, 'load_duration': 126091459, 'prompt_eval_count': 27, 'prompt_eval_duration': 949630458, 'eval_count': 57, 'eval_duration': 3206213790, 'logprobs': None, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--c1d809a9-f997-4e32-a736-7816ec46cbee-0', usage_metadata={'input_tokens': 27, 'output_tokens': 57, 'total_tokens': 84})

In [18]:
SOURCE = """
Old Ship Saloon 2023 quarterly revenue numbers:
Q1: $174782.38
Q2: $467372.38
Q3: $474773.38
Q4: $389289.23
"""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 'You are a helpful assistant. Use the following context when responding:\n\n{context}.'),
    ("human", "{question}")
])

rag_chain = rag_prompt | chat_model | StrOutputParser()

rag_chain.invoke({
    "question": "What was the Old Ship Saloon's total revenue in Q1 2023?",
    "context": SOURCE
})

"The Old Ship Saloon's total revenue for Q1 2023 was $174,782.38."

## Debugging

In [41]:
from langchain_core.globals import set_debug

set_debug(True)

from datetime import date

prompt = ChatPromptTemplate.from_messages([
    ("system", 
 """IMPORTANT INSTRUCTION:
 You MUST answer using the date: {current_date}.
 Never invent your own date.
 If asked for the current date, always return using: {current_date}.
 """),
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser()

chain.invoke({
    "question": "What is the current date?",
    "current_date": date.today()
})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "System: IMPORTANT INSTRUCTION:\n You MUST answer using the date: 2025-11-28.\n Never invent your own date.\n If asked for the current date, always return using: 2025-11-28.\n \nHuman: What is the current date?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOllama] [1.20s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "2025-11-28.",
        "generation_info": {
          "model": "llama3.1",
          "created_at": "2025-11-28T03:02:25.341074Z",
          "done": true,
          "done_reason": "stop",
          "total_duration": 1191999333,
          "loa

'2025-11-28.'

Using Async

In [43]:
# Turn off debug mode for clarity
set_debug(False)

stream = chain.astream_events({
    "question": "What is the current date?",
    "current_date": date.today()
}, version="v1")

async for event in stream:
    print(event)
    print("-----")

{'event': 'on_chain_start', 'run_id': '3db24b20-970c-4303-8d86-30b380c636b9', 'name': 'RunnableSequence', 'tags': [], 'metadata': {}, 'data': {'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 11, 28)}}, 'parent_ids': []}
-----
{'event': 'on_prompt_start', 'name': 'ChatPromptTemplate', 'run_id': 'd7c6ad48-fb64-4b28-b6ef-e5dc7fca8739', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 11, 28)}}, 'parent_ids': []}
-----
{'event': 'on_prompt_end', 'name': 'ChatPromptTemplate', 'run_id': 'd7c6ad48-fb64-4b28-b6ef-e5dc7fca8739', 'tags': ['seq:step:1'], 'metadata': {}, 'data': {'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 11, 28)}, 'output': ChatPromptValue(messages=[SystemMessage(content='IMPORTANT INSTRUCTION:\n You MUST answer using the date: 2025-11-28.\n Never invent your own date.\n If asked for the current date, always retu

In [44]:
# Sign up at <https://smith.langchain.com/>
# Set environment variables

# import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = Access.LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = "default"

chain.invoke({
  "question": "What is the current date?",
  "current_date": date.today()
})

'2025-11-28.'